In [104]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from torch.utils.data import DataLoader

import torch_loader
reload(torch_loader)
reload(preprocessing)
import preprocessing as pre

from importlib import reload

In [100]:


# default prediction window = 60, sliding_window=30
ds = torch_loader.WebTrafficDataset('data', 'train_1')


train_loader = DataLoader(ds,
                         batch_size=1,
                         shuffle=True,
                         num_workers=4)

for batch_idx, data in enumerate(train_loader):
    print(batch_idx)
    print(data)
    break

Loading pickle...
Done!
For lag: 365 nan percent is 0.153
For lag: 91 nan percent is 0.038
Done loading and preprocessing set!
0
{'tseries': 

Columns 0 to 9 
 5.2311  4.7791  4.9273  5.5872  5.4072  5.0173  5.0106  5.1985  4.9628  5.2781

Columns 10 to 19 
 5.4161  5.7961  5.4161  5.2417  4.9488  5.9738  5.9558  6.3154  6.3852  6.0234

Columns 20 to 29 
 5.7398  5.7900  5.5334  5.5491  5.6490  5.7104  5.3033  5.2149  5.2257  4.9698

Columns 30 to 39 
 4.7185  5.1358  5.1818  4.7449  4.8040  5.2883  5.2679  5.2311  5.7301  5.5683

Columns 40 to 49 
 5.2470  5.2311  5.1059  4.9053  4.8978  4.9836  5.0039  4.9273  4.3307  4.6728

Columns 50 to 59 
 4.4067  4.8675  5.2149  5.2883  4.9200  4.6250  4.6540  4.4427  4.4886  4.7449

Columns 60 to 69 
 4.9488  4.5539  4.1897  4.0254  4.2485  4.4308  4.7005  4.5951  4.2341  4.0073

Columns 70 to 79 
 3.8067  4.4886  4.5109  4.5109  4.5109  3.6889  3.9703  3.9120  3.7377  3.9703

Columns 80 to 89 
 4.6634  4.6728  4.1431  4.1897  4.1897  3.7842  

In [107]:
(res, tar) = pre.series_supervised_np(ds[0]['tseries'], lags=10, drop_nan=True)

tar.shape

(345,)

In [108]:
class Sequential(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, bias=True):
        super(Sequential, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.bias = bias
        
        self.gru1 = nn.GRUCell(self.input_size, self.hidden_size, bias=self.bias)
        self.gru2 = nn.GRUCell(self.hidden_size, self.hidden_size, bias=self.bias)
        self.linear = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, input, future=0):
        hidden_t1 = Variable(torch.zeros(input.size(0), self.hidden_size).double(),
                            requires_grad=False)
        hidden_t2 = Variable(torch.zeros(input.size(0), self.hidden_size).double(),
                            requires_grad=False)
        
        outputs = []
        for i, input_t in enumerate(input.chunk(input.size(0), dim=1)):
            # input should be (seq_len, batch_size, input_size) for LSTM
            # (batch, input_size) for GRUCell
            hidden_t1 = self.gru1(input_t, hidden_t1)
            hidden_t2 = self.gru2(hidden_t1, hidden_t2)
            output = self.linear(hidden_t2)
            outputs += [output]
            
        for i in range(future):
            hidden_t1 = self.gru1(output, hidden_t1)
            hidden_t2 = self.gru2(hidden_t1, hidden_t2)
            output = self.linear(hidden_t2)
            outputs += [output]
            
        outputs = torch.stack(outputs, 1).squeeze(2)
        out = outputs[-1]
        return out
input_size = 1 # number of features. 1 for just tseries
hidden_size = 30
batch_size = 30

model = Sequential(input_size, hidden_size, batch_size)
model.double()
model

Sequential (
  (gru1): GRUCell(1, 30)
  (gru2): GRUCell(30, 30)
  (linear): Linear (30 -> 1)
)

In [147]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.2):
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=1,
            dropout=dropout,
            bidirectional=False)
        self.fc1 = nn.Linear(self.hidden_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        batch_size = x.size(0)
        seq_len = x.size(1)
        
        x = x.view(seq_len, batch_size, -1)
        
        h0 = Variable(torch.zeros(seq_len, batch_size, self.hidden_size).double())
        c0 = Variable(torch.zeros(seq_len, batch_size, self.hidden_size).double())
        
        out, (ht, ct) = self.rnn(x, (h0, c0))
        
        out = out[-1]
        out = self.fc1(out)
        out = self.bn1(out)
        #out = self.relu(out)
        out = F.dropout(out, training=self.training)
        out = self.fc2(out)
        return out
    
input_size = 1 # number of features. 1 for just tseries
hidden_size = 30

model = LSTM(input_size, hidden_size)
model.double()
model

LSTM (
  (rnn): LSTM(1, 30, dropout=0.2)
  (fc1): Linear (30 -> 30)
  (bn1): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True)
  (fc2): Linear (30 -> 1)
  (relu): ReLU ()
)

In [149]:
tseries = (ds[10000]['tseries'].reshape(-1))
(tseries, target) = pre.series_supervised_np(tseries, lags=10, drop_nan=True)
tseries = Variable(torch.from_numpy(tseries))
out = model(tseries)
out

Variable containing:
-0.3039
-0.1010
 0.4717
 0.2621
-0.0829
 0.1347
-0.0653
 0.3231
 0.3139
-0.0099
 1.1192
 0.4854
 0.1369
 1.1210
-0.0098
 0.1521
 0.2832
 0.2568
 0.0024
-0.2461
 0.1698
-0.0019
 0.5822
-0.2708
 0.2001
 0.3001
 0.4438
-0.0468
-0.1867
-0.2638
-0.3405
 0.3832
 0.1530
 0.1851
 0.4477
 0.1686
 0.0416
-0.2460
-0.1712
 0.0494
 0.1101
 0.3905
 0.3892
 0.4086
 0.1440
-0.0900
-0.0505
-0.3584
 0.4311
 0.0006
 0.4333
-0.0501
-0.0887
 0.0893
 0.0638
-0.0988
-0.1819
 0.0572
 0.1163
 0.0274
 0.4229
-0.2425
 0.0163
-0.0130
-0.1448
-0.1074
 0.1139
 0.2515
 0.2237
 0.0692
 0.3213
 0.7680
 0.1913
-0.1075
-0.3192
 0.2462
 0.2421
 0.2896
 0.1369
-0.3404
 0.3318
-0.1796
-0.0613
-0.3879
 0.2594
 0.3578
 0.2507
 0.2042
 0.1083
-0.2094
 0.1002
-0.1328
-0.1301
 0.2676
-0.0762
 0.2070
-0.0556
-0.2748
 0.0145
-0.0426
-0.1525
-0.0551
 0.2095
 0.4746
 0.2810
-0.0132
-0.2295
-0.2211
-0.0529
-0.1315
-0.0722
 0.1816
 0.0675
-0.0859
 0.0371
-0.2208
-0.0615
 0.0484
-0.5368
-0.6026
 0.3281
 0.3508
 0.

In [ ]:
losses = []

def train(epoch):
    model.train()
    criterion = nn.MSELoss()
    optimizer = optim.RMSprop(model.parameters(), lr=1e-3, 
                           weight_decay=1e-3, momentum=0.2)
    
    for batch_idx, data in enumerate(train_loader):
        tseries = data['tseries'].numpy().reshape(-1)
        (tseries, target) = pre.series_supervised_np(np.nan_to_num(tseries), 
                                                     lags=10, drop_nan=True)
        
        input = Variable(torch.from_numpy(tseries))
        target = Variable(torch.from_numpy(target))
        
        optimizer.zero_grad()
        output = model(input)
        
        loss = criterion(output, target)
        losses.append(loss)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
        if batch_idx == 10000:
            break

train(1)
losses

Train Epoch: 1 [0/145063 (0%)]	Loss: 1.844445
Train Epoch: 1 [10/145063 (0%)]	Loss: 12.832309
Train Epoch: 1 [20/145063 (0%)]	Loss: 1.017194
Train Epoch: 1 [30/145063 (0%)]	Loss: 13.301526
Train Epoch: 1 [40/145063 (0%)]	Loss: 0.769080
Train Epoch: 1 [50/145063 (0%)]	Loss: 9.796189
Train Epoch: 1 [60/145063 (0%)]	Loss: 1.587932
Train Epoch: 1 [70/145063 (0%)]	Loss: 0.846916
Train Epoch: 1 [80/145063 (0%)]	Loss: 7.098526
Train Epoch: 1 [90/145063 (0%)]	Loss: 0.723116
Train Epoch: 1 [100/145063 (0%)]	Loss: 7.783759
Train Epoch: 1 [110/145063 (0%)]	Loss: 1.550737
Train Epoch: 1 [120/145063 (0%)]	Loss: 6.247850
Train Epoch: 1 [130/145063 (0%)]	Loss: 6.351440
Train Epoch: 1 [140/145063 (0%)]	Loss: 4.385904
Train Epoch: 1 [150/145063 (0%)]	Loss: 6.422185
Train Epoch: 1 [160/145063 (0%)]	Loss: 10.031444
Train Epoch: 1 [170/145063 (0%)]	Loss: 9.267537
Train Epoch: 1 [180/145063 (0%)]	Loss: 2.392793
Train Epoch: 1 [190/145063 (0%)]	Loss: 2.063300
Train Epoch: 1 [200/145063 (0%)]	Loss: 1.402950


In [ ]:
# plot predictions

In [ ]:
# plot losses

In [ ]:
# plot learning curves

In [ ]:
# cross validation